## Setup

In [79]:
import pandas as pd
import sqlite3 as s3
import pyodbc
import warnings
warnings.simplefilter('ignore')

# DB = {'servername': 'NOAH\SQLEXPRESS01',
#       'database': '4.3 db'}

# export_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER=' + DB['servername'] + 
#                             ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
# export_cursor = export_conn.cursor()
# export_cursor

go_sales = s3.connect('go_sales.sqlite')
go_staff = s3.connect('go_staff.sqlite')
go_crm = s3.connect('go_crm.sqlite')

product = pd.read_sql_query('SELECT * FROM product', go_sales)
product_type = pd.read_sql_query('SELECT * FROM product_type', go_sales)
sales_staff = pd.read_sql_query('SELECT * FROM sales_staff', go_sales)
sales_branch = pd.read_sql_query('SELECT * FROM sales_branch', go_sales)
staff_manager = pd.read_sql_query('SELECT * FROM sales_staff', go_staff)
order_method = pd.read_sql_query('SELECT * FROM order_method', go_sales)
return_reason = pd.read_sql_query('SELECT * FROM return_reason', go_sales)
retailer_contact = pd.read_sql_query('SELECT * FROM retailer_contact', go_crm)

### PRODUCT

In [73]:
merged = pd.merge(product, product_type, left_on='PRODUCT_TYPE_CODE', how='inner', right_on='PRODUCT_TYPE_CODE')
new_product = pd.DataFrame(merged[['PRODUCT_NUMBER', 'PRODUCT_NAME', 'DESCRIPTION', 'INTRODUCTION_DATE', 'PRODUCT_TYPE_CODE', 'PRODUCTION_COST', 'MARGIN', 'PRODUCT_IMAGE', 'LANGUAGE']])
new_product

# for index, row in new_product.iterrows():
#     try:
#         query = f"INSERT INTO product VALUES ({row['PRODUCT_NUMBER']}, '{row['PRODUCT_NAME']}', '{row['DESCRIPTION']}', '{row['INTRODUCTION_DATE']}', '{row['PRODUCT_TYPE_CODE']}', '{row['PRODUCTION_COST']}', '{row['MARGIN']}', '{row['PRODUCT_IMAGE']}', '{row['LANGUAGE']}')"
#         export_cursor.execute(query)
#     except pyodbc.Error:
#         print(query)

# export_conn.commit()
# export_cursor.close()

,PRODUCT_NUMBER,PRODUCT_NAME,DESCRIPTION,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE
0,1,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",15-2-2011,1,4,.33,P01CE1CG1.jpg,EN
1,10,TrailChef Utensils,"Spoon, fork and knife set made of a light yet ...",15-2-2011,1,10,.4,P10CE1CG1.jpg,EN
2,100,Insect Bite Relief,The Insect Bite Relief helps the itching and s...,15-2-2011,17,3,.5,P100OP4FA17.jpg,EN
3,101,Hailstorm Steel Irons,Iron is 17-4 stainless steel. Shafts are grap...,15-12-2019,18,305.54,.43,P101GE5IR18.jpg,EN
4,102,Hailstorm Titanium Irons,Made entirely of pure titanium. The ultimate i...,10-12-2019,18,380.95,.51,P102GE5IR18.jpg,EN
...,...,...,...,...,...,...,...,...,...
110,95,Sun Shield,"PABA free sunscreen, SPF 30, poison oak and iv...",15-2-2011,16,3,.5,P91OP4SS16.jpg,EN
111,96,Compact Relief Kit,A personal first aid kit is recommended for ev...,15-2-2011,17,16.43,.28,P96OP4FA17.jpg,EN
112,97,Deluxe Family Relief Kit,A complete medical kit suitable for families w...,5-3-2013,17,25,.28,P96OP4FA17.jpg,EN
113,98,Calamine Relief,Use the Calamine Relief for allergic skin reac...,15-2-2011,17,3,.5,P98OP4FA17.jpg,EN


### SALES_STAFF

In [74]:
merged = pd.merge(staff_manager, sales_branch, left_on='SALES_BRANCH_CODE', how='inner', right_on='SALES_BRANCH_CODE')
new_staff = pd.DataFrame(merged[['SALES_STAFF_CODE', 'WORK_PHONE', 'FAX', 'EMAIL', 'FIRST_NAME', 'LAST_NAME', 'POSITION_EN', 'EXTENSION', 'DATE_HIRED', 'SALES_BRANCH_CODE', 'MANAGER_CODE']])
new_staff

# for index, row in new_staff.iterrows():
#     try:
#         query = f"INSERT INTO sales_staff VALUES ({row['SALES_STAFF_CODE']}, '{row['WORK_PHONE']}', '{row['FAX']}', '{row['EMAIL']}', '{row['FIRST_NAME']}', '{row['LAST_NAME']}', '{row['POSITION_EN']}', '{row['EXTENSION']}', '{row['DATE_HIRED']}', '{row['SALES_BRANCH_CODE']}', '{row['MANAGER_CODE']}')"
#         export_cursor.execute(query)
#     except pyodbc.Error:
#         print(query)

# export_conn.commit()
# export_cursor.close()

,SALES_STAFF_CODE,WORK_PHONE,FAX,EMAIL,FIRST_NAME,LAST_NAME,POSITION_EN,EXTENSION,DATE_HIRED,SALES_BRANCH_CODE,MANAGER_CODE
0,100,+358(0)17 - 433 127,+358(0)17 - 433 129,TSavolainen@grtd123.com,Tuomas,Savolainen,Level 2 Sales Representative,825,23-Jul-1998 12:00:00 AM,31,18
1,101,82-2-778-6587,82-2-778-6586,CKim@grtd123.com,Chang-ho,Kim,Level 2 Sales Representative,1228,15-Dec-1998 12:00:00 AM,32,87
2,102,82-2-778-6587,82-2-778-6586,JChoi@grtd123.com,Jung-ho,Choi,Level 3 Sales Representative,2641,27-Nov-1998 12:00:00 AM,32,87
3,103,886-2-2975-1988,886-2-2989-4978,KKwao@grtd123.com,Kwei-Ping,Kao,Level 2 Sales Representative,1541,07-Oct-1998 12:00:00 AM,33,87
4,104,886-2-2975-1988,886-2-2989-4978,CFon@grtd123.com,Chin-Tsai,Fang,Level 3 Sales Representative,2311,08-May-1996 12:00:00 AM,33,87
...,...,...,...,...,...,...,...,...,...,...,...
97,95,+35 94 322 3540,+35 91 548 1637,TIglesias@grtd123.com,Tomás,Iglesias,Level 2 Sales Representative,325,19-Feb-1998 12:00:00 AM,29,18
98,96,+55 (11) 344-4444,+55 (11) 333-2223,BLopes@grtd123.com,Bayard,Lopes,Branch Manager,3490,12-Aug-1997 12:00:00 AM,30,71
99,97,+55 (11) 344-4444,+55 (11) 333-2223,VBarros@grtd123.com,Viviam,Barros,Level 3 Sales Representative,3364,10-Feb-2001 12:00:00 AM,30,96
100,98,+55 (11) 344-4444,+55 (11) 333-2223,EGuimaraes@grtd123.com,Eduardo,Guimarães,Level 2 Sales Representative,1442,14-Feb-1999 12:00:00 AM,30,96


### ORDER_METHOD

In [75]:
new_order_method = order_method[['ORDER_METHOD_CODE', 'ORDER_METHOD_EN']]
new_order_method

# for index, row in new_order_method.iterrows():
#     try:
#         query = f"INSERT INTO order_method VALUES ({row['ORDER_METHOD_CODE']}, '{row['ORDER_METHOD_EN']}')"
#         export_cursor.execute(query)
#     except pyodbc.Error:
#         print(query)

# export_conn.commit()
# export_cursor.close()

,ORDER_METHOD_CODE,ORDER_METHOD_EN
0,1,Fax
1,2,Telephone
2,3,Mail
3,4,E-mail
4,5,Web
5,7,Sales visit
6,8,Special


### RETURN_REASON

In [77]:
new_return_reason = return_reason[['RETURN_REASON_CODE', 'RETURN_DESCRIPTION_EN']]
new_return_reason

# for index, row in new_return_reason.iterrows():
#     try:
#         query = f"INSERT INTO return_reason VALUES ({row['RETURN_REASON_CODE']}, '{row['RETURN_DESCRIPTION_EN']}')"
#         export_cursor.execute(query)
#     except pyodbc.Error:
#         print(query)

# export_conn.commit()
# export_cursor.close()

,RETURN_REASON_CODE,RETURN_DESCRIPTION_EN
0,1,Defective product
1,2,Incomplete product
2,3,Wrong product ordered
3,4,Wrong product shipped
4,5,Unsatisfactory product


### RETAILER_CONTACT

In [84]:
new_retailer_contact = retailer_contact[['RETAILER_CONTACT_CODE', 'FAX', 'E_MAIL', 'RETAILER_SITE_CODE', 'FIRST_NAME', 'LAST_NAME', 'JOB_POSITION_EN', 'EXTENSION', 'GENDER']]
new_retailer_contact

# for index, row in new_retailer_contact.iterrows():
#     try:
#         last_name = row['LAST_NAME'].replace("'", "''")
#         query = f"INSERT INTO return_reason VALUES ({row['RETAILER_CONTACT_CODE']}, '{row['FAX']}', '{row['EMAIL']}', '{row['RETAILER_SITE_CODE']}', '{row['FIRST_NAME']}', '{last_name}', '{row['JOB_POSITION_EN']}', '{row['EXTENSION']}', '{row['GENDER']}')"
#         export_cursor.execute(query)
#     except pyodbc.Error:
#         print(query)

# export_conn.commit()
# export_cursor.close()

,RETAILER_CONTACT_CODE,FAX,E_MAIL,RETAILER_SITE_CODE,FIRST_NAME,LAST_NAME,JOB_POSITION_EN,EXTENSION,GENDER
0,10,1 (514) 741-1610,FSmith@legolfeurinc.com,14,Frank,Smith,Chief Purchaser,2489,M
1,100,1 (702) 369-2274,Y2883@emertxe.com,85,Amanda,Yates,Stock Manager,None,F
2,101,1 (702) 882-0978,S8832@emertxe.com,84,Paula,Schmidt,Assistant Purchaser,None,F
3,102,1 (713) 524-3215,T8839@emertxe.com,88,Micheal,Tao,District Manager,445,M
4,103,1 (305) 557-5478,Murphy@eyedimensions3.com,92,Jack,Murphy,District Manager,None,M
...,...,...,...,...,...,...,...,...,...
386,95,1 (423) 266-4721,G1289@emertxe.com,87,Jennifer,Gomes,Site Manager,None,F
387,96,1 (304) 343-4560,M0019@emertxe.com,91,Timothy,Moon,Site Assistant Manager,None,M
388,97,1 (206) 362-6645,F9943@emertxe.com,90,Paul,Fischer,Stock Manager,None,M
389,98,1 (703) 549-4896,W3219@emertxe.com,89,Derek,Wilkes,Site Assistant Manager,None,M
